In [4]:
import importlib
import sys
sys.path.append('..')

import miann.data._data
from miann.data._data import MPPData
from miann.constants import DATA_DIR
importlib.reload(miann.data._data)

import logging
logging.basicConfig()
import os


In [5]:
mpp_data = MPPData.from_data_dir(os.path.join(DATA_DIR, '184A1_hannah_unperturbed/I09'))

In [7]:
mpp_data.metadata

,mapobject_id,plate_name,well_name,well_pos_y,well_pos_x,tpoint,zplane,label,is_border,mapobject_id_cell,...,label_cell,is_border_cell,is_mitotic,is_mitotic_labels,is_polynuclei_HeLa,is_polynuclei_HeLa_labels,is_polynuclei_184A1,is_polynuclei_184A1_labels,is_SBF2_Sphase_labels,is_SBF2_Sphase
0,373530,plate01,I09,0,0,0,0,1,0,373501,...,1,1,0.0,NaN,0.0,NaN,0.0,NaN,NaN,1.0
3,373533,plate01,I09,0,0,0,0,4,0,373504,...,4,1,0.0,NaN,0.0,NaN,1.0,NaN,NaN,1.0
5,373535,plate01,I09,0,0,0,0,6,0,373506,...,6,0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,1.0
6,373536,plate01,I09,0,0,0,0,7,0,373507,...,7,0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0
7,373537,plate01,I09,0,0,0,0,8,0,373508,...,8,0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,248104,plate01,I09,5,4,0,0,39,0,248059,...,39,1,0.0,NaN,0.0,NaN,0.0,NaN,NaN,1.0
1116,248105,plate01,I09,5,4,0,0,40,0,248060,...,40,1,1.0,NaN,1.0,NaN,0.0,NaN,NaN,0.0
1117,248106,plate01,I09,5,4,0,0,41,0,248061,...,41,1,0.0,NaN,0.0,NaN,0.0,NaN,NaN,1.0
1119,248108,plate01,I09,5,4,0,0,43,0,248063,...,43,1,0.0,NaN,0.0,NaN,0.0,NaN,NaN,1.0


In [9]:
df = mpp_data.channels

In [13]:
import pandas as pd

In [16]:
channels = pd.read_csv(os.path.join(mpp_data.data_dir, 'channels.csv'), names=['channel_id', 'name'], index_col=0)

In [22]:
import numpy as np

In [27]:
np.load?

Signature: np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
Docstring:
Load arrays or pickled objects from ``.npy``, ``.npz`` or pickled files.

.. warning:: Loading files that contain object arrays uses the ``pickle``
             module, which is not secure against erroneous or maliciously
             constructed data. Consider passing ``allow_pickle=False`` to
             load data that is known not to contain object arrays for the
             safer handling of untrusted sources.

Parameters
----------
file : file-like object, string, or pathlib.Path
    The file to read. File-like objects must support the
    ``seek()`` and ``read()`` methods. Pickled files require that the
    file-like object support the ``readline()`` method as well.
mmap_mode : {None, 'r+', 'r', 'w+', 'c'}, optional
    If not None, then memory-map the file, using the given mode (see
    `numpy.memmap` for a detailed description of the modes).  A
    memory-mapped array i

In [30]:
np.load(os.path.join(mpp_data.data_dir, f'mpp.npy'), mmap_mode='r')

memmap([[152, 114, 113, ..., 118, 131, 289],
        [143, 112, 122, ..., 119, 123, 312],
        [137, 115, 130, ..., 109, 120, 273],
        ...,
        [167, 116, 180, ..., 115, 133, 272],
        [141, 106, 177, ..., 117, 121, 231],
        [142, 110, 193, ..., 114, 126, 188]], dtype=int32)

In [20]:
channels.reset_index(drop=True)

,name
0,00_EU
1,01_CDK9_pT186
2,01_PABPC1
3,02_CDK7
4,03_CDK9
5,03_RPS6
6,05_GTF2B
7,05_Sm
8,07_POLR2A
9,07_SETD1A
